In [ ]:
from functools import partial
from multiprocessing import cpu_count
from multiprocessing.pool import Pool

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from distance_determination import estimate_dist, get_current_freq
from interpolation.interpolate import interpolate, sklearn_interpolations
from interpolation.metrics import get_RMSE
from interpolation.wsinterp import wsinterp
from simul.signals.augment import signal_add_noise
from simul.utilities.data import load_experiment
from simul.vis.dist_probs import vis_dist_probs
from simul.vis.signals import get_vis_df, vis_signals, vis_signals2d, get_fft_df
from tqdm.auto import tqdm, trange

%load_ext autoreload
%autoreload 2

In [ ]:
sample_rate_full = 1000  #  Samples per s
sin_freq = 5  # Hz
sin_omega = (2*np.pi)*sin_freq
sin_ts = np.arange(0, 5, 1/sample_rate_full)
# sin_vals = np.sin(sin_omega * sin_ts)

signal_vals_full = np.exp(1j * sin_omega * sin_ts)



df = get_vis_df(
    sin_ts,
    np.array([signal_vals_full]),
    np.array([signal_vals_full]),
#     *interp_signals,
#     n=20000,
    freqs=[0],
)
vis_signals2d(df)

In [ ]:
def prune_normal(signal, rate=0.05, std=0.1):
#     ratio: 0.1
    res = np.full(signal.shape, np.NaN, dtype=complex)
    grid_ids = np.arange(0, signal.shape[0], 1/rate)
    grid_deltas = np.random.normal(0, std, grid_ids.shape)
    grid_ids += grid_deltas
    grid_ids = grid_ids[(grid_ids < signal.shape[0]-1)&((grid_ids > 0))]
#     print(np.rint(grid_ids).astype(int))
    res[np.rint(grid_ids).astype(int)] = signal[np.rint(grid_ids).astype(int)]
    return res

signal_vals_pruned = prune_normal(signal_vals_full, rate = 0.1, std=2)
df = get_vis_df(
    sin_ts,
    np.array([signal_vals_full]),
    np.array([signal_vals_pruned]),
#     *interp_signals,
#     n=20000,
    freqs=[0],
)
vis_signals2d(df)


In [ ]:

def randomized_sinc_interp(x:np.ndarray, xp:np.ndarray, fp:np.ndarray, sigma_coeff=0.8,
                            randomize=False, left=None, right=None)->np.ndarray:

    Tn = (xp[-1] - xp[0])/(xp.shape[0]-1)
#     print(xp.shape, xp[0], xp[1], Tn)
    xp_regular = np.arange(xp[0], xp[-1]+Tn*0.1, Tn)
    
    xp_deltas = xp - xp_regular
    if(randomize):
        spread_0 = np.mean(np.square(xp_deltas))
        xp_result = xp_regular + np.random.normal(0, spread_0*sigma_coeff, xp_regular.shape)
    xp_result = xp_regular + xp_deltas * sigma_coeff 

    # shape = (nxp, nx), nxp copies of x data span axis 1
    u = np.resize(x, (len(xp), len(x)))
    # Must take transpose of u for proper broadcasting with xp.
    # shape = (nx, nxp), v(xp) data spans axis 1
    # v = (xp - u.T) / (Tn)
    v = (xp_result - u.T) / (Tn)
    # shape = (nx, nxp), m(v) data spans axis 1
    m =   fp * np.sinc(v)
    # Sum over m(v) (axis 1)
    fp_at_x = np.sum(m, axis=1)

    # Enforce left and right
    if left is None:
        left = fp[0]
    fp_at_x[x < xp[0]] = left
    if right is None:
        right = fp[-1]
    fp_at_x[x > xp[-1]] = right

    return fp_at_x

def interpolate_rand(signals: np.ndarray, sigma:float = 0.8, randomize=False):
    interp_signal = []

    x = np.arange(signals.shape[1])
#     for signal in tqdm(signals):
    for signal in signals:
        idx = np.where(~np.isnan(signal))[0]
        interp_signal.append(randomized_sinc_interp(x, x[idx], signal[idx], sigma, randomize))

    return np.array(interp_signal)

In [ ]:
def run_test(signal_vals, rate, std, randomize=False):
    signals_vals_pruned = prune_normal(signal_vals, rate, std)

    interp_signals = [
        (interpolate_rand(np.array([signals_vals_pruned]), sigma, randomize=randomize), f"Sigma:{sigma}") for sigma in [0, 0.1, 0.2, 0.5, 0.7, 0.8, 1]
    ]
    for interp in interp_signals:
        mse = np.abs((np.square(interp[0][0,:] - signal_vals))).mean(axis=0)/np.max(np.abs(signal_vals))
#                 mse = np.sqrt(np.abs((np.square(interp[0][0,:] - signal_vals))).mean(axis=0))/np.max(np.abs(signal_vals))

        print(interp[1],":        ",  mse)
    return vis_signals2d(
        get_vis_df(
            sin_ts,
            np.array([signal_vals]),
            np.array([signals_vals_pruned]),
            *interp_signals,
    #         n=20000,
            freqs=[0],
    ))

In [ ]:
for std in (1, 10, 100, 200):
    print(std)
    run_test(signal_vals_full, 0.2, std)


In [ ]:
for std in (1, 10, 100, 200):
    print(std)
    run_test(signal_vals_full, 0.2, std, randomize=True)


In [ ]:
for std in (1, 10, 100, 200, 500):
    print("Spread:", std)
    run_test(signal_vals_full, 0.045, std)

In [ ]:
for std in ( 500, 1000, 2000):
    print("Spread:", std)
    run_test(signal_vals_full, 0.1, std)

In [ ]:
for rate in (0.5, 0.2, 0.1, 0.05, 0.02):
    run_test(signal_vals_full, rate, 0.2/rate)


In [ ]:
sample_rate_full = 1000  #  Samples per s
sin_freq = 5  # Hz
sin_freq2 = 1.1  # Hz

sin_omega = (2*np.pi)*sin_freq
sin_omega2 = (2*np.pi)*sin_freq2

sin_ts = np.arange(0, 5, 1/sample_rate_full)
# sin_vals = np.sin(sin_omega * sin_ts)

signal_vals_two_full = np.exp(1j * sin_omega * sin_ts)+np.exp(1j * sin_omega2 * sin_ts)*0.4



df = get_vis_df(
    sin_ts,
    np.array([signal_vals_two_full]),
    np.array([signal_vals_two_full]),
#     *interp_signals,
#     n=20000,
    freqs=[0],
)
vis_signals2d(df)

In [ ]:
for std in (1, 10, 100, 200):
    print(std)
    run_test(signal_vals_two_full, 0.2, std)

In [ ]:
for std in (1, 10, 100, 200, 500):
    print("Spread:", std)
    run_test(signal_vals_two_full, 0.045, std)

In [ ]:
for rate in (0.5, 0.2, 0.1, 0.05, 0.02):
    run_test(signal_vals_full, rate, 0.2/rate)